# FInDe

## Availables banks
- Cajamar
- Openbank
- Unicaja
- BBVA

## Fist step
Put your csv or xls files inside ```accounts/current/<bank_name>/file``` and ```accounts/savings/<bank_name>/file```

## Requisites

In [ ]:
!pip install plotly
!pip install numpy
!pip install pandas
!pip install xlrd
!pip install openpyxl

## Code

### Imports

In [ ]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd
import pandas as pd

py.offline.init_notebook_mode(connected=True)

### Constant and settings

In [ ]:
CURRENT = "current"
SAVING = "saving"
ACCOUNTTYPES  = [CURRENT, SAVING]
SUPPORTED_BANKS = {
    "unicaja": "Unicaja",
    "cajamar": "Cajamar",
    "openbank": "Openbank",
    "bbva": "BBVA"
}
CURRENCY = '€'

### Process CSV, XLS and XLSX

In [ ]:
def processCSV(file, splitToken, bank): #TODO Move to pandas
    fecha = []
    saldo = []
    movimientos = []
    first = True
    
    if bank == 'cajamar':
        with open(file) as f:
            for line in f:
                if first is False:
                    tokens = line.split(splitToken)
                    fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                    saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                    movimientos.append(float(tokens[len(tokens)-2].replace('.','').replace(',','.').replace(' ', '')))
                first = False

    return (saldo, fecha, movimientos)

In [ ]:
def processXLS(file, bank):   
    balance = []
    date = []
    movements = []
    
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        movements = first_sheet.col_values(3)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date() for i in range(0, len(date_int))]
    
    if bank == 'openbank':
        # TODO Improve and read like sheets
        df = pd.read_html(file)
        df = df[0].dropna(axis=0, thresh=4)
        
        date_str = df.iloc[1:,1]
        date = [dt.datetime.strptime(x, '%d/%m/%Y').date() for x in date_str]
        
        balance_str = df.iloc[1:,9]
        balance = []
        for x in balance_str:
            if ',' in x:
                balance.append(float(x.replace('.','').replace(',','.')))
            else:
                balance.append(float((x[:-2] + ',' + x[-2:]).replace('.','').replace(',','.')))
        
        movements_str = df.iloc[1:,7]
        movements = []
        for x in movements_str:
            if ',' in x:
                movements.append(float(x.replace('.','').replace(',','.')))
            else:
                movements.append(float((x[:-2] + ',' + x[-2:]).replace('.','').replace(',','.')))

    return (balance, date, movements)

In [ ]:
def processXLSX(file, bank):   
    balance = []
    date = []
    movements = []
    
    if bank == 'bbva':
        df = pd.read_excel(file)

        datetime = df.iloc[4:,1]
        date = [x.date() for x in datetime]

        balance = df.iloc[4:,7]

        movements = df.iloc[4:,5]

    return (balance, date, movements)

### Load files

In [ ]:
def sortDataFiles(bankdata):
    for accountType in ACCOUNTTYPES:
        for bankname in bankdata[accountType]:
            if "movements" in bankdata[accountType][bankname]:
                (bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"], bankdata[accountType][bankname]["movements"]) = zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"], bankdata[accountType][bankname]["movements"])))
            else:
                (bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"]) = zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"])))
            
    return bankdata

In [ ]:
def loadDataFiles():    
    bankdata = {}
    for accountType in ACCOUNTTYPES:
        path = "accounts/{}/".format(accountType)
        directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
        bankdata[accountType] = {}
        for bankname in directories:
            bankdata[accountType][bankname] = {"date":[], "balance":[], "movements":[]}
            files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
            #files.sort(key=lambda x: os.path.getmtime(x))
            print(f"Total files found for {bankname}: {len(files)}")
            for datafile in files:
                extension = os.path.splitext(datafile)[1]
                if extension == ".xls":
                    #print("{} - {}".format(bankname, datafile))
                    (balance, date, movements) = processXLS(datafile, bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
                    bankdata[accountType][bankname]["movements"].extend(movements)
                if extension == '.xlsx':
                    #print("{} - {}".format(bankname, datafile))
                    (balance, date, movements) = processXLSX(datafile, bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
                    bankdata[accountType][bankname]["movements"].extend(movements)
                if extension == ".csv":
                    #print("{} - {}".format(bankname, datafile))
                    (balance, date, movements) = processCSV(datafile, '\t', bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
                    bankdata[accountType][bankname]["movements"].extend(movements)

    return sortDataFiles(bankdata)

In [ ]:
def getIntervalDates(bankdata):
    minDate = ''
    maxDate = ''
    first = True
    
    for accountType in ACCOUNTTYPES:
        for bank in bankdata[accountType]:
            dates = np.array(bankdata[accountType][bank]['date'])
            
            if first:
                minDate = dates.min()
                maxDate = dates.max()
                
                first = False
            else:
                if minDate > dates.min():
                    minDate = dates.min()

                if maxDate < dates.max():
                    maxDate = dates.max()

    return (minDate, maxDate)

In [ ]:
def fillEmpty(bankdata, maxDate):
    for accountType in ACCOUNTTYPES:
        for bank in bankdata[accountType]:
            if bankdata[accountType][bank]['date'][-1] != maxDate:
                bankdata[accountType][bank]["balance"] = bankdata[accountType][bank]["balance"] + (bankdata[accountType][bank]["balance"][-1],)
                bankdata[accountType][bank]["date"] = bankdata[accountType][bank]["date"] + (maxDate,)
                
    return bankdata

### Traces

#### Extra

In [ ]:
def daterange(d1, d2):
    return (d1 + dt.timedelta(days=i) for i in range((d2 - d1).days + 1))

def get_trace_sum_balances(bankdata):
    sum_balances = []
    for accountType in ACCOUNTTYPES: # Account type
        for bank in bankdata[accountType]: # Bank name
            movements = bankdata[accountType][bank]['movements']
            dates = bankdata[accountType][bank]['date']
            balances = bankdata[accountType][bank]['balance']
            sum_account = {}
            for date, movement, balance in zip(dates, movements, balances):
                if (date not in sum_account):
                    sum_account[date] = balance
                elif((balance < sum_account[date] and movement < 0) or (balance > sum_account[date] and movement > 0)):
                    sum_account[date] = balance

            sum_balances.append(sum_account)
    
    total = {}
    (ini, fin) = getIntervalDates(bankdata)
    for b in sum_balances:
        last = 0
        for d in daterange(ini, fin):
            if d in b:
                last = b[d]
                if d in total:
                    total[d] += b[d]
                else:
                    total[d] = b[d]
            else:
                if d in total:
                    total[d] += last
                else:
                    total[d] = last
                
    
    dates = total.keys()
    balances = total.values()
    
    (dates, balances) = zip(*sorted(zip(dates, balances)))
    
    trace = go.Scatter(
        x = dates,
        y = balances,
        name = "All Accounts - Amount: " + format(balances[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".") + CURRENCY,
        mode = 'lines',
        line = dict ( width = 4 )
    )

    return (max(balances), trace)

#### Build

In [ ]:
def buildTraces(bankdata, min_trace = False, mean_trace = False, max_trace = False):
    data = []
    min_amount = 0
    max_amount = 0
    
    for accountType in ACCOUNTTYPES: # Account type
        for bank in bankdata[accountType]: # Bank name
            dates = np.array(bankdata[accountType][bank]['date'])
            movements = np.array(bankdata[accountType][bank]['movements'])
            balances = np.array(bankdata[accountType][bank]['balance'])

            total = 0
            for date, movement, balance in zip(dates, movements, balances):
                #Obtener total mejorado
                if (total == 0):
                    total = balance
                elif((balance < total and movement < 0) or (balance > total and movement > 0)):
                    total = balance
            
            bankName = '(' + accountType + ') ' + SUPPORTED_BANKS[bank]

            trace_main = go.Scatter(
                x = dates,
                y = balances,
                name = bankName + ': Saldo ' + str(format(total, ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + CURRENCY,
                #line = dict(
                #    color = 'green'
                #),
                mode = 'lines'
            )
            data.append(trace_main)

            if max_trace:
                trace_max = go.Scatter(
                    x = dates,
                    y = [balances.max() for f in dates],
                    name = bankName + ': Saldo máximo',
                    #visible = 'legendonly',
                    #hoverinfo = 'name',
                    line = dict(
                        #color = 'cyan',
                        width = 4,
                        dash = 'dot'
                    )
                )
                data.append(trace_max)

            if mean_trace:
                trace_mean = go.Scatter(
                    x = dates,
                    y = [balances.mean() for f in dates],
                    name = bankName + ': Saldo medio',
                    #hoverinfo = 'none',
                    line = dict(
                        #color = 'magenta',
                        width = 4,
                        dash = 'dashdot'
                    )
                )
                data.append(trace_mean)

            if min_trace:
                trace_min = go.Scatter(
                    x = dates,
                    y = [balances.min() for f in dates],
                    name = bankName + ': Saldo mínimo',
                    line = dict(
                        #color = 'red',
                        width = 4,
                        dash = 'dot'
                    )
                )
                data.append(trace_min)
                
            # Extra
            if balances.max() > max_amount:
                max_amount = balances.max()
    
    max_amount, sum_trace = get_trace_sum_balances(bankdata)
    data.append(sum_trace)
    
    return (data, min_amount, max_amount)

In [ ]:
def buildPiggy(bankdata):
    traces = []
    
    for bank in bankdata[SAVING]: # Bank name
        balances = np.array(bankdata[SAVING][bank]['balance'])
        
        trace = go.Bar(
            x = ['Piggy'],
            y = [balances[-1]],
            name = SUPPORTED_BANKS[bank]
        )
        
        traces.append(trace)
        
    return traces

In [ ]:
def buildProfit(bankdata):
    data = []

    for bank in bankdata[CURRENT]:
        dates = bankdata[CURRENT][bank]['date']
        movements = bankdata[CURRENT][bank]['movements']

        profit = {}
        for date, movement in zip(dates, movements):
            key = str(date.month) + '/' + str(date.year)

            if key in profit:
                profit[key] += float(movement)
            else:
                profit[key] = float(movement)
            
        months = []
        profits = []
        for key, value in profit.items():
            months.append(dt.datetime.strptime(key, '%m/%Y').date())
            profits.append(value)
        
        trace = go.Bar(
            x = months,
            y = profits,
            name = "Profit for {}".format(SUPPORTED_BANKS[bank])
        )
        data.append(trace)
        
    return data

In [ ]:
def buildIncomesExpenses(bankdata):
    data = []

    for bank in bankdata[CURRENT]:
        dates = bankdata[CURRENT][bank]['date']
        movements = bankdata[CURRENT][bank]['movements']

        incomes = {}
        expenses = {}
        for date, movement in zip(dates, movements):
            key = str(date.month) + '/' + str(date.year)

            if float(movement) > 0:
                if key in incomes:
                    incomes[key] += float(movement)
                else:
                    incomes[key] = float(movement)
            else:
                if key in expenses:
                    expenses[key] += float(movement)
                else:
                    expenses[key] = float(movement)
            
        months_x = []
        incomes_y = []
        for key, value in incomes.items():
            months_x.append(dt.datetime.strptime(key, '%m/%Y').date())
            incomes_y.append(value)
        
        trace = go.Bar(
            x = months_x,
            y = incomes_y,
            name = "Incomes for {}".format(SUPPORTED_BANKS[bank])
        )
        data.append(trace)
        
        months_x = []
        expenses_y = []
        for key, value in expenses.items():
            months_x.append(dt.datetime.strptime(key, '%m/%Y').date())
            expenses_y.append(value)
        
        trace = go.Bar(
            x = months_x,
            y = expenses_y,
            name = "Expenses for {}".format(SUPPORTED_BANKS[bank])
        )
        data.append(trace)
        
    return data

### Plot

In [ ]:
def plot_general(bankdata, minDate, maxDate):
    (data, _, _) = buildTraces(bankdata)
    layout = go.Layout(title = 'Amount ' + minDate.strftime("%m/%d/%Y") + ' - ' + maxDate.strftime("%m/%d/%Y"),
                  xaxis = dict(title = 'Date'),
                  yaxis = dict(title = 'Amount (' + CURRENCY + ')'),
                  showlegend = True
    )

    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig, filename='styled-line')

In [ ]:
def plot_piggy(bankdata):
    data = buildPiggy(bankdata)
    layout = go.Layout(
        title = 'Saving',
        barmode='stack'
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='stacked-bar')

In [ ]:
def plot_super_view(bankdata, minDate, maxDate):
    (data, min_amount, max_amount) = buildTraces(bankdata)
    piggyData = buildPiggy(bankdata)
    for piggyTrace in piggyData:
        newTrace = go.Bar(
            x = piggyTrace.x,
            y = piggyTrace.y,
            name = piggyTrace.name,
            xaxis = 'x2',
            yaxis = 'y2'
        )
        data.append(newTrace)
    layout = go.Layout(
        xaxis=dict(
            domain=[0, 0.9],
            title = 'Date',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1 month',
                         step='month',
                         stepmode='backward'),
                    dict(count=3,
                         label='3 months',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6 months',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='1 year',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible = True
            ),
            type='date'
        ),
        yaxis=dict(
            range=[min_amount, max_amount],
            title = 'Amount (' + CURRENCY + ')'
        ),
        xaxis2=dict(
            domain=[0.9, 1]
        ),
        yaxis2=dict(
            anchor='x2',
            range=[min_amount, max_amount],
            showticklabels=False
        ),
        title = 'Super view ' + minDate.strftime("%m/%d/%Y") + ' - ' + maxDate.strftime("%m/%d/%Y"),
        barmode='stack'
    )
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='side-by-side-subplot')

In [ ]:
def plot_profit(bankdata):
    data = buildProfit(bankdata)
    layout = go.Layout(
        title = 'Monthly profit',
        barmode ='group',
        xaxis = dict(title = 'Date'),
        yaxis = dict(title = 'Amount (' + CURRENCY + ')')
    )
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='grouped-bar')

In [ ]:
def plot_incomesExpenses(bankdata):
    data = buildIncomesExpenses(bankdata)
    layout = go.Layout(
      xaxis = dict(title = 'Date'),
      yaxis = dict(title = 'Amount (' + CURRENCY + ')'),
      barmode = 'relative',
      title = 'Incomes and expenses'
    )
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='barmode-relative')

In [ ]:
def plot_incomesExpensesProfits(bankdata):
    data = buildIncomesExpenses(bankdata)
    
    dataProfit = buildProfit(bankdata)
    
    # Profits per bank
    for barProfit in dataProfit:
        trace_profit = go.Scatter(
            x = barProfit.x,
            y = barProfit.y,
            name = barProfit.name,
            mode = 'markers'
        )
        data.append(trace_profit)
    
    # Total profits
    totalProfits = {}
    for barProfit in dataProfit:
        for date, value in zip(barProfit.x, barProfit.y):
            key = date.strftime("%m/%Y")
            if key in totalProfits:
                totalProfits[key] += value
            else:
                totalProfits[key] = value
    
    xs = []
    ys = []
    for key, value in totalProfits.items():
        xs.append(dt.datetime.strptime(key, '%m/%Y').date())
        ys.append(value)
    
    trace_profit = go.Scatter(
        x = xs,
        y = ys,
        name = 'Profit',
        mode = 'markers',
        marker = dict(
            size = 10,
            line = dict(
                width = 2,
            )
        )
    )
    data.append(trace_profit)
    
    layout = go.Layout(
      xaxis = dict(
          title = 'Date',
          rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1 month',
                     step='month',
                     stepmode='backward'),
                dict(count=3,
                     label='3 months',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6 months',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1 year',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
      ),
      yaxis = dict(title = 'Amount (' + CURRENCY + ')'),
      barmode = 'relative',
      title = 'Incomes, expenses and profit'
    )
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename='barmode-relative')

### Main

In [ ]:
bankdata = loadDataFiles() # Load files
(minDate, maxDate) = getIntervalDates(bankdata) # Get dates interval
bankdata = fillEmpty(bankdata, maxDate) # Fill missing data with the same value

#plot_general(bankdata, minDate, maxDate)
#plot_piggy(bankdata)
plot_super_view(bankdata, minDate, maxDate)
#plot_profit(bankdata)
#plot_incomesExpenses(bankdata)
plot_incomesExpensesProfits(bankdata)